# Autonomous Trading Agents - With 0xMONK from FereAI

<a target="_blank" href="https://colab.research.google.com/github/fere-ai/agentic-examples/blob/main/python/trading-agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook demonstrates the ease of building autonomous trading agents. It's super easy using the `0xMONK` APIs from FereAI.

- [Developer Docs](https://docs.fereai.xyz/docs/api/api-introduction)
- [Get Your API Key](https://docs.google.com/forms/d/e/1FAIpQLScFAWfT5u2kwKx8tQkkL9AfdP8NIlNIn6tXJUQcRipZEUZokA/viewform)
- [#dev-help on our Discord](https://discord.com/invite/3fsm5XJNW8)

In [1]:
import requests
import json
from pprint import pprint
from tabulate import tabulate
from uuid import  UUID
import logging

In [12]:
fere_api_key = "1234" # @param {type: "string"}
fere_user_id = "d9a486fa-8dd0-4593-b2f6-9443d3a0abd6" # @param {type: "string"}

The main class `DiscipleClient` has all the methods needed to interact with Disciple APIs

In [14]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

class DiscipleClient:
    # base_url = "https://api.fereai.xyz/ta/"
    base_url = "http://localhost:8001/"

    def __init__(self, api_key, user_id):
        """
        Initializes the FereAI class with an API key.

        Args:
            api_key (str): The API key for authenticating requests.
            user_id (str): The user ID for identifying the user.
        """
        self.api_key = api_key
        self.user_id = user_id

    @property
    def api_headers(self):
        """
        Generates headers for API requests.

        Returns:
            dict: The headers including the Authorization key.
        """
        return {
            "X-FRIDAY-KEY": self.api_key,
            "Content-Type": "application/json",
            "X-FERE-USERID": self.user_id,
        }

    def create_agent(
      self,
      fere_user_id: UUID,
      name: str | None=None,
      description: str | None=None,
      persona: str | None=None,
      data_source: str | None=None,
      decision_prompt_pool: str | None=None,
      decision_prompt_portfolio: str | None=None,
      twitter_username: str | None=None,
      fc_username: str|None=None,
      simulation:bool=False,
      simulation_initial_usd: float=1000.0,
      simulation_initial_native: float=1.0,
      max_investment_per_session: float | None=None,
      stop_loss: float | None=0.6,
      trailing_stop_loss: float| None=None,
      take_profit: float|None=0.5,
      chain: str ='base',
      mode: str ='MANUAL',
      parent_ids: list[UUID] | None=None,
      investment_for_buy: float | None=None,
      investment_for_buy_usd: float|None=10.0,
      ):
      """
      Creates a new trading agent.

      Args:
          fere_user_id (str): The user ID.
          name (str): The name of the agent.
          description (str): A description of the agent.
          persona (str): The persona of the agent.
          data_source (str): The data source for the agent.
          decision_prompt_pool (str): The decision prompt pool.
          decision_prompt_portfolio (str): The decision prompt portfolio.
          twitter_username (str, optional): Twitter username.
          fc_username (str, optional): FC username.
          simulation (bool, optional): Whether to run in simulation mode.
          simulation_initial_usd (int, optional): Initial USD for simulation.
          simulation_initial_native (int, optional): Initial native currency for simulation.
          max_investment_per_session (float, optional): Max investment per session.
          stop_loss (float, optional): Stop loss percentage.
          trailing_stop_loss (float, optional): Trailing stop loss percentage.
          take_profit (float, optional): Take profit percentage.
          chain (str, optional): Blockchain chain.
          mode (str, optional): The agent mode.
          parent_ids (list, optional): The list of parent agent IDs.
          investment_for_buy (float, optional): Investment for buy.
          investment_for_buy_usd (float, optional): Investment for buy in USD.

      Returns:
          dict or None: The created agent data or None on failure.
      """
      url = self.base_url + "agent/"
      payload = json.dumps({
          "user_id": fere_user_id,
          "name": name,
          "description": description,
          "persona": persona,
          "chain": chain,
          "data_source": data_source,
          "decision_prompt_pool": decision_prompt_pool,
          "decision_prompt_portfolio": decision_prompt_portfolio,
          "twitter_username": twitter_username,
          "fc_username": fc_username,
          "dry_run": simulation,
          "dry_run_initial_usd": simulation_initial_usd,
          "dry_run_initial_native": simulation_initial_native,
          "max_investment_per_session": max_investment_per_session,
          "stop_loss": stop_loss,
          "trailing_stop_loss": trailing_stop_loss,
          "take_profit": take_profit,
          "mode": mode,
          "parent_ids": parent_ids,
          "investment_for_buy": investment_for_buy,
          "investment_for_buy_usd": investment_for_buy_usd,
      })

      response = requests.put(url, headers=self.api_headers, data=payload, timeout=10)

      if response.status_code == 200:
          data = response.json()
          return data
      else:
          print(response.text)
          return None

    def get_agent(self, agent_id: UUID | str) -> dict:
        """
        Fetches an agent by ID.

        Args:
            agent_id (str): The agent ID.

        Returns:
            dict or None: The agent data or None on failure.
        """
        url = f"{self.base_url}agent/{agent_id}"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def fetch_disciples(self, fere_user_id):
        """
        Fetches disciples for a user ID.

        Args:
            fere_user_id (str): The user ID.

        Returns:
            list: A list of disciples or an empty list on failure.
        """
        url = f"{self.base_url}agent/user/{fere_user_id}/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            data = response.json()
            print(f"Disciples Created: {len(data)}\n")
            for disciple in data:
                print(f"Name: {disciple['name']}")
                print(f"ID: {disciple['id']}\n")
            return data
        else:
            print(f"Failed to fetch disciples: {response.text}")
            return []

    def get_portfolio(self, disciple_agent_id):
        """
        Fetches portfolio for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The portfolio data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/portfolio/"
        response = requests.get(url, headers=self.api_headers, timeout=60)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def get_holdings(self, disciple_agent_id):
        """
        Fetches holdings for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            list or None: The holdings data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/holdings/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(response.text)
            return None

    def get_trades(self, disciple_agent_id):
        """
        Fetches trades for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The trades data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/trades/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            pprint(response.text)
            return None

    def get_trade_recommendations(self, disciple_agent_id):
        """
        Fetches trade recommendations for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The trade recommendations data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/decisions/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def get_optimal_gains(self, disciple_agent_id, page=1, page_size=10, sort_by_latest_signal=True, sort_column='noticed_at', sort_order='desc'):
        """
        Fetches optimal gains for a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The optimal gains data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/calls/"
        params = {
            "page": page,
            "page_size": page_size,
            "sort_by_latest_signal": sort_by_latest_signal,
            "sort_column": sort_column,
            "sort_order": sort_order
        }
        response = requests.get(url, headers=self.api_headers, params=params, timeout=20)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def get_decisions(self, disciple_id: UUID):
        """Gets the decisions of the agent.
        """
        url = f"{self.base_url}agent/{disciple_id}/decisions/"
        response = requests.get(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def update_agent(self, disciple_id, **kwargs):
        """
        Updates an agent.

        Args:
            disciple_id (str): The agent ID.
            **kwargs: Parameters for updating the agent.

        Returns:
            dict or None: The updated agent data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_id}/"
        payload = json.dumps(kwargs)
        response = requests.patch(url, headers=self.api_headers, data=payload, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def schedule_sell(self, disciple_id: UUID, ca: str, quantity: int | str = "all"):
      """
      Schedules a sell for a holding.

      Args:
          disciple_id (str): The disciple ID.
          ca (str): The CA to sell.
          quantity (str): The quantity to sell.

      Returns:
          dict or None: The response data or None on failure.
      """
      url = f"{self.base_url}/agent/{disciple_id}/sell/{ca}/{quantity}/"
      payload = {
      }

      response = requests.request("POST", url, headers=self.api_headers, json=payload, timeout=10)

      if response.status_code == 200:
        return response.json()
      else:
        logger.debug(f"Got response code {response.status_code} with message {response.text}")

    def schedule_buy(self, disciple_id: UUID, ca: str, quantity: int | None, value_usd: float | None, slippage: float = 1.0):
      """
      Schedules a buy for a symbol.

      Args:
          disciple_id (str): The disciple ID.
          symbol (str): The symbol to buy.
          quantity (str): The quantity to buy.
          value_usd (str): The value in USD to buy.
          slippage (str): The slippage percentage.

      Returns:
          dict or None: The response data or None on failure.
      """
      url = f"{self.base_url}/agent/{disciple_id}/buy/"
      payload = {
          "ca": ca,
          "quantity": quantity,
          "value_usd": value_usd,
          "slippage": slippage,
      }

      response = requests.request("POST", url, headers=self.api_headers, json=payload, timeout=10)

      if response.status_code == 200:
        return response.json()
      else:
        print(response.text)

    def check_task_status(self, task_id):
      """
      Checks the status of a task.

      Args:
          task_id (str): The task ID.

      Returns:
          dict or None: The task status data or None on failure.
      """
      url = f"{self.base_url}/task/status/{task_id}/"

      payload = {}

      response = requests.request("GET", url, headers=self.api_headers, data=payload, timeout=10)

      if response.status_code == 200:
        return response.json()
      else:
        print(response.text)

    def delete_disciple(self, disciple_agent_id):
        """
        Deletes a disciple agent.

        Args:
            disciple_agent_id (str): The agent ID.

        Returns:
            dict or None: The response data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_agent_id}/"
        response = requests.delete(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def add_parents(self, disciple_id: UUID, parents: list[UUID]):
        """
        Adds parents to the agent.

        Args:
            parents (list): The list of parent agent IDs.

        Returns:
            dict or None: The response data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_id}/parents/"
        payload = json.dumps({"parent_ids": parents})
        response = requests.patch(url, headers=self.api_headers, data=payload, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def delete_parents(self, disciple_id: UUID, parent_ids: list[UUID]):
        """
        Deletes parents from the agent.

        Returns:
            dict or None: The response data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_id}/parents/"
        payload = json.dumps({"parent_ids": parent_ids})
        response = requests.delete(url, headers=self.api_headers, data=payload, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None

    def schedule_sync(self, disciple_id: UUID):
        """
        Schedules a sync for the agent.

        Args:
            disciple_id (str): The agent ID.

        Returns:
            dict or None: The response data or None on failure.
        """
        url = f"{self.base_url}agent/{disciple_id}/sync/"
        response = requests.patch(url, headers=self.api_headers, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            print(response.text)
            return None



## 0. Create a Disciple Instance

In [16]:
disciple = DiscipleClient(fere_api_key, fere_user_id)


## 1. Create an Agent

Instructions Examples: https://docs.fereai.xyz/docs/product/0xMONK/#coin-decision-instructions


In [28]:
name = "Aron Alpha Agent" # @param {type: "string"}
description = "A loyal and trusted disciple of MONK" # @param {type: "string"}
persona = "A loyal and trusted disciple of MONK" # @param {type: "string"}
data_source = "trending" # @param ["trending", "latest", "virtuals_s"]
decision_prompt_pool = "You are a pro memecoin trader on Solana. Use your understanding of memcoins, market psychology and on-chain metrics to identify coins with high growth potentials.  MISSION: Make an informed and calculated judgement on weather to buy, hold, sell or pass on a memecoin.  GOAL: Your GOAL is to generate consistent profits over a **longer-term period (3 - 7 days)** by blending **momentum trading, volatility management, and adaptive feedback loops**.  STRATEGY: ### 1. **Initial Assessment for Entry**    - **Assess Pool Age:**      - If the pool was created recently (within 1-2 days), it suggests early-stage momentum. Prioritize memecoins that show strong early trading interest, as indicated by high **volume in the first 24 hours**.     - **Check FDV vs. Market Cap Ratio:**      - **FDV close to Market Cap:** Indicates potential bullishness with less risk of dilution. This supports entering a longer-term trade.      - **FDV significantly higher than Market Cap:** Indicates risk of dilution; proceed cautiously with tighter risk management.     - **Review Price Change Trends (up to 24hr):**      - Enter trades where there is a **consistent upward price change** across intervals (5m, 1hr, 6hr, 24hr), supported by rising volume.      - For a **bullish entry**, ensure positive price change across multiple intervals, particularly in 1hr, 6hr, and 24hr.  ### 2. **Incorporate Volatility Analysis**    - **Use ATR to Define Entry and Exit Ranges:**      - Calculate the **Average True Range (ATR)** to set entry and stop-loss levels. Wider ATR suggests higher potential rewards, but also increased risk.      - Set initial stop-loss levels at **1.5-2x the ATR** to avoid premature exits due to regular volatility.     - **Volatility Breakout Entry:**      - Enter long positions when the price breaks above recent volatility bands (e.g., Bollinger Bands), accompanied by strong volume over 6hr intervals.      - Avoid entering trades during extreme volatility spikes unless there is consistent follow-through in volume and price over longer intervals (6hr or more).  ### 3. **Adaptive Position Sizing**    - **Start with Smaller Initial Positions:**      - Given the high volatility, begin with a smaller position size to minimize risk. As the trend confirms (positive price and volume over 12hr), gradually increase the position size.     - **Adjust Position Based on Recent Trade Success:**      - If recent trades have been profitable, **increase position size by 10-20%** to capitalize on momentum.      - If recent trades have had high losses, **reduce position size** and tighten stops to avoid further losses.  ### 4. **Feedback Loop with Last Trades**    - **Analyze Trade Patterns:**      - Review the **last 5 trades** to identify common triggers for success or failure (e.g., rapid volume surges, resistance breakouts, or social media-driven momentum).      - If the **last profitable trades** had similar conditions (e.g., positive price change after volume spikes), replicate those entry conditions for prolonged trades.     - **Use Historical Exit Points for Guidance:**      - If past trades reached a **profit target of 10-20%** within 12-24 hours, set similar targets and be ready to adjust based on current momentum.      - If the last loss resulted from holding too long despite declining volume, be prepared to exit sooner in similar conditions.  ### 5. **Mid-Trade Management for 12-36 Hours**    - **Reassess at 12hr and 24hr Intervals:**      - Every 12 hours, evaluate whether the momentum is sustained: check volume, price change, and transaction patterns. If the volume remains strong and price trends are positive, maintain the position.     - **Trailing Stop-Loss:**      - Implement a **trailing stop-loss** based on the 6hr ATR to lock in profits as the price moves up. This allows you to capture gains while keeping the potential for further upside.      - Adjust the trailing stop if momentum remains strong after 24 hours, allowing for an extended hold.  ### 6. **Longer-Term Hold Strategy (Up to 7 Days)**    - **Monitor 24hr and 6hr Volume Trends:**      - If 24hr volume continues to rise, hold the position for up to 3 days to maximize gains.      - For holds longer than 3 days, ensure consistent positive price change in the 24hr interval and sustained volume increases.     - **Identify Extended Trends or Reversals:**      - Use **12hr and 24hr moving averages (MA)** as benchmarks:        - If the price consistently stays above the 12hr MA, it indicates sustained bullish momentum—hold longer.        - If the price drops below the 12hr MA, consider reducing position size or exiting partially.  ### 7. **Exit Criteria for 3-7 Days**    - **Profit Targets:**      - Set multiple profit targets (e.g., at +10%, +20%, and +30%).      - Use the **last 5 successful trades** to determine the most effective profit levels.     - **Volatility and Volume Drop:**      - Exit if there’s a significant **drop in 6hr or 24hr volume**, as it signals a decline in momentum.      - Exit earlier if there’s a sudden spike in **sell transactions** over 6hr intervals, indicating a potential trend reversal.    - **OHLCV Analysis:**     - If the **Open-High-Low-Close-Volume (OHLCV)** pattern shows consistent lower highs and lower lows over 24hr intervals, consider exiting to avoid further losses.     - If the OHLCV pattern remains bullish with higher highs and higher lows, consider holding the position for an extended period.     - You have OHLCV data in durations of 1 minute, 1 hour and 1 day. Access all of them carefully.  ### Summary of Strategy Workflow 1. **Entry:**    - Pool age, FDV vs. Market Cap, consistent positive price changes, and volatility breakout.  2. **Position Sizing:**    - Start small, scale up as momentum confirms, and adapt based on recent trades.  3. **Mid-Trade Management:**    - Reassess every 12hr, use trailing stops, and adjust based on ATR.  4. **Longer Hold:**    - Use 24hr volume and MA trends to sustain the trade.  5. **Exit:**    - Based on profit targets, volume drops, increased sell transactions, or external events." # @param {type: "string"}
decision_prompt_portfolio = "You are a seasoned crypto memecoin portfolio manager on Solana. Your only focus and job is keep increasing profits for yourself.  You are managing a portfolio of memecoins.  You are about to take following decision on your portfolio. You must analyze each of them critically. Specially look at the amount you're about to invest in each coin. Look at the coin's FDV and mcap and analyse if it's the best way to invest.  Also, you can rebalance your portfolio by selling some coins and buying new ones. You can also hold some coins if you think they have potential to grow further." # @param {type: "string"}
twitter_username = "0xmonk" # @param {type: "string"}
fc_username = "0xmonk" # @param {type: "string"}
max_investment_per_session = None # @param {type:"slider", min:0, max:1, step:0.01}
stop_loss = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
trailing_stop_loss = 0.3 # @param {type:"slider", min:0, max:1, step:0.01}
take_profit = 1.0 # @param {type:"number"}

# @markdown ### Simulation specific settings
simulation = False # @param {type: "boolean"}
simulation_initial_usd = None # @param {type: "number"}
simulation_initial_native = 0.1 # @param {type: "number"}
chain = "solana" # @param ["base", "solana", "arbitrum"]
mode = "MANUAL" # @param ["MANUAL", "COPY_TRADE", "AUTONOMOUS"]
parent_ids = [] # ["1857242a-e3df-4fc9-8de5-2a111663789c"]
investment_for_buy = None # @param {type: "number"}
investment_for_buy_usd = 10.0 # @param {type: "number"}

In [29]:
agent = disciple.create_agent(fere_user_id, name,description,persona,data_source,
             decision_prompt_pool, decision_prompt_portfolio, twitter_username,
             fc_username, simulation, simulation_initial_usd, simulation_initial_native,
             max_investment_per_session, stop_loss, trailing_stop_loss,
             take_profit, chain, mode, parent_ids, investment_for_buy,
             investment_for_buy_usd)

pprint(f"Created with ID: {agent['id']} and name: {agent['name']}")

pprint(agent)

('Created with ID: 83a3232a-a7e1-4b25-b8fc-f09d2e58b571 and name: Aron Alpha '
 'Agent')
{'chain': 'SOLANA',
 'data_source': 'trending',
 'decision_prompt_pool': None,
 'decision_prompt_portfolio': None,
 'description': 'A loyal and trusted disciple of MONK',
 'dry_run': False,
 'dry_run_initial_native': None,
 'dry_run_initial_usd': 14.432,
 'fc_username': '0xmonk',
 'id': '83a3232a-a7e1-4b25-b8fc-f09d2e58b571',
 'investment_for_buy': None,
 'investment_for_buy_usd': 10.0,
 'is_active': True,
 'max_investment_per_session': None,
 'mode': 0,
 'name': 'Aron Alpha Agent',
 'parent_ids': None,
 'parents': [],
 'persona': 'A loyal and trusted disciple of MONK',
 'stop_loss': 0.5,
 'take_profit': 1.0,
 'trailing_stop_loss': 0.3,
 'twitter_username': '0xmonk',
 'user_id': 'd9a486fa-8dd0-4593-b2f6-9443d3a0abd6',
 'wallet': {'address': '6w5kzpd6MrXiJT3X9hxgDqqBD8NXHpFREGv4xEvmjbYE',
            'pvt_key': '3a6cJuXBX8Exip7pbtk3qm1ZzBpMTJwQjvxSK2w2qmbwFFBgN47ccjGUQfWX4tpXkCDgFiTT8821kew8UuatPzQJ

## 2. Get Disciple Agents from a given User

In [18]:
pprint(disciple.fetch_disciples(fere_user_id))

Disciples Created: 3

Name: Aron Alpha Agent
ID: 83a3232a-a7e1-4b25-b8fc-f09d2e58b571

Name: PP Alpha Monk
ID: 82bff902-88ad-43ea-a8e2-433c1f899e1a

Name: spicy-designer
ID: 2428457f-3cae-4ee7-b8d5-876bb06d00d9

[{'address': 'FCM9FQwU4XpqLTfB5QgscNmKjZwfCkM54YFAV1zcLFtD',
  'chain': 'SOLANA',
  'data_source': 'trending',
  'decision_prompt_pool': None,
  'decision_prompt_portfolio': None,
  'description': 'A loyal and trusted disciple of MONK',
  'dry_run': False,
  'dry_run_initial_usd': 14.432,
  'fc_username': '0xmonk',
  'id': '83a3232a-a7e1-4b25-b8fc-f09d2e58b571',
  'investment_for_buy': None,
  'investment_for_buy_usd': 10.0,
  'is_active': True,
  'max_investment_per_session': None,
  'mode': 0,
  'name': 'Aron Alpha Agent',
  'parents': [],
  'persona': 'A loyal and trusted disciple of MONK',
  'stop_loss': 0.5,
  'take_profit': 1.0,
  'trailing_stop_loss': 0.3,
  'twitter_username': '0xmonk',
  'user_id': 'd9a486fa-8dd0-4593-b2f6-9443d3a0abd6'},
 {'address': 'G8K7wN9wRbHzZ3t2

## 3. Get portfolio

In [31]:
disciple_agent_id = "82bff902-88ad-43ea-a8e2-433c1f899e1a" # @param {type: "string"}


In [32]:
portfolio = disciple.get_portfolio(disciple_agent_id)

curr_amount = portfolio['curr_realised_usd'] + portfolio['curr_unrealised_usd']
profit = curr_amount - portfolio['start_usd']
profit_percent = (profit / portfolio['start_usd']) * 100 if portfolio['start_usd'] > 0 else 0

pprint(portfolio)
pprint(f"Profit %: {profit_percent}")

{'agent_id': '83a3232a-a7e1-4b25-b8fc-f09d2e58b571',
 'curr_realised_native': 0.0,
 'curr_realised_usd': 0.0,
 'curr_unrealised_native': 0.0,
 'curr_unrealised_usd': 0.0,
 'dry_run': False,
 'id': '2e4c2533-8be9-4fce-993e-2b79983f0850',
 'start_native': 0.0,
 'start_time': '2025-03-24T14:02:44.650205Z',
 'start_usd': 0.0}
'Profit %: 0'


## 4. Get Holdings

In [53]:
holdings = disciple.get_holdings(disciple_agent_id)

print(f"Total holdings: {len(holdings)}")

_h = []
for h in holdings:
  # pprint(h)
  _h.append([h['id'], h['base_address'], h['pool_name'], h['tokens_bought'],
             h['tokens_bought'] * h['buying_price_usd'],
             h['tokens_bought'] * h['buying_price_native'],
             h['tokens_bought'] * h.get('curr_price_usd', 0),
             h['profit_abs_usd'],
             h['profit_abs_native']
             ])

headers = ["ID","Base Address", "Pool Name", "Tokens Bought", "Invested $", "Invested Native", "Current $", "Current Native", "Profit $", "Profit Native"]
print(tabulate(_h, headers=headers))

Total holdings: 4
ID                                    Base Address                                  Pool Name           Tokens Bought    Invested $    Invested Native    Current $    Current Native      Profit $
------------------------------------  --------------------------------------------  ----------------  ---------------  ------------  -----------------  -----------  ----------------  ------------
bdc99fbf-b4a7-4101-a9f5-9b004ccfbbe2  ENSfJsbKLzH3jyT53UnzGm2okLdA87d8fGyZabyCpump  SignalGate / SOL        1883.6      0.999868           0.00705065   0.556534         -0.443334     -0.00301919
2fbb6f45-aedb-4105-82c3-27b921922229  5LJMJyR8MtAkbtpf8kFUV7S9oFG3xaGDdcnFxYt9pump  FAT / SOL                 47.8525   1.00238            0.00716538   0.838959         -0.163425     -0.00106643
60682577-6bc9-4799-ac0e-4c322bdbe60c  Gi2ce3Eu6dt8iAS2pa8hLsoudSEvew477SVJkHUipump  MAG / SOL                  1e-06    4.82594e-10        3.71222e-12  2.17435e-10      -2.65159e-10  -2.11099e-12
0922

## 5. Get Trades


In [52]:

trades = disciple.get_trades(disciple_agent_id)
print(trades)

_t = []
for t in trades:
  _t.append([
    t['base_address'],
    t['pool_name'],
    'buy' if t['decision'] == 1 else 'sell',
    t['in_amount'],
    t['out_amount'],
    t['price_usd'],
    t['created_at'],
    t['txn'],
    t['confidence'],
    # t['reason'],
    ])

headers =  ['Base Address', 'Pool Name', 'Decision', 'In Amount', 'Out Amount', 'Price USD', 'Created At', 'Txn', 'Confidence', 'Reason']

print(f"Total Trades: {len(trades)}")

# filter_ca = ["znv3FZt2HFAvzYf5LxzVyryh3mBXWuTRRng25gEZAjh", "C7heQqfNzdMbUFQwcHkL9FvdwsFsDRBnfwZDDyWYCLTZ"]
# _t = [t for t in _t if t[0] in filter_ca]

print(tabulate(_t, headers=headers))

[{'created_at': '2025-03-26T20:14:36.496000Z', 'agent_id': '82bff902-88ad-43ea-a8e2-433c1f899e1a', 'base_address': 'EyzgnBfHGe9hh169B8993muBVcoeURCnSgPbddBeSybo', 'pool_name': 'IPLR / SOL', 'decision': 0, 'price_usd': 0.000666348217130726, 'price_sol': 4.82546322782769e-06, 'in_amount': 680649990000000.0, 'out_amount': 0.002623829, 'gas_fee': 127000.0, 'jito_fee': 300000.0, 'other_amount_threshold': 2927157.0, 'reason': 'Sell decision executed manually.', 'future_action': "Re-evaluate if there's an entry criteria", 'profit_sol': None, 'profit_usd': None, 'profit_percentage': None, 'txn': 'https://solscan.io/tx/3WP2JMJvEWLTfA99rTTwzMc7EGHbFPUdNW3kgPNGxRexiZsvJtDCLBEqtbGeKnkSZdzs8QVWVhM6yDVWPxT2xwX1', 'dry_run': False, 'confidence': 100.0}, {'created_at': '2025-03-26T14:50:48Z', 'agent_id': '82bff902-88ad-43ea-a8e2-433c1f899e1a', 'base_address': '5LJMJyR8MtAkbtpf8kFUV7S9oFG3xaGDdcnFxYt9pump', 'pool_name': 'FAT / SOL', 'decision': 1, 'price_usd': 0.020947387509594042, 'price_sol': 0.00014

In [ ]:
8.598059


## 6. Get Optimal gains

In [151]:
disciple_agent_id = 'e5e0155e-f8ac-4885-8cbe-27899e3a2325'
disciple.get_optimal_gains(disciple_agent_id)

{'total_count': 780,
 'buy_decisions': [{'id': '836c31b7-6750-444e-841f-801b67b5b360',
   'agent_id': 'e5e0155e-f8ac-4885-8cbe-27899e3a2325',
   'base_address': 'BAPYqqFWNnmihtui9PNSAbnsBfPuErZdkdUDyyKepump',
   'pool_address': '3Rd51pEEHM9QwGKi6yjnxdWdQNo372cRY5bC9wWqM4sE',
   'pool_name': 'Panther / SOL',
   'token_name': 'Pink Panther',
   'symbol': 'Panther',
   'profile_pic': 'https://assets.geckoterminal.com/4s7yq330odk5txjs7h5k6iuxcasp',
   'noticed_at': '2025-03-18T22:43:25.761379Z',
   'peaked_at': '2025-03-18T23:10:00Z',
   'notice_price_usd': 0.000844945080381256,
   'notice_price_native': 6.83590238370804e-06,
   'notice_mcap': 844920.96351183,
   'price_usd': 0.0013066072520642,
   'price_native': 1.04211776364986e-05,
   'peak_mcap': 1043642.74822608,
   'curr_price_usd': 9.31181881554277e-05,
   'curr_price_native': 7.35325323556158e-07,
   'curr_mcap': 93115.5303268457,
   'profit_per_usd': 54.6381276608692,
   'profit_per_native': 52.4477245511195,
   'dry_run': True,


## 7. Sell a held token manually

In [39]:
ca = "EyzgnBfHGe9hh169B8993muBVcoeURCnSgPbddBeSybo"  # @param {type: "string"}
# ca = "9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump"
quantity = "all" # @param {type: "string"}

A sell is executed as a background task. Upon calling the sell API, it returns a task ID. This task ID will be used to check the task status.

In [51]:

task = disciple.schedule_sell(disciple_agent_id, ca, quantity)

pprint(task)

{'task_id': 'e1d1e317-d2c0-4ddb-80e3-93278b131cc5'}


Now once we have the task ID, we can check the status of the task.

In [44]:
status = disciple.check_task_status(task['task_id'])

pprint(status)

{'result': {},
 'status': 'FAILURE',
 'task_id': '077de050-f8f5-4fe8-83f1-37f8d340cb11'}


Depending on the status, show the updates to user.

## 8. Buy a coin manually

In [ ]:
solana_ca = "9gyfbPVwwZx4y1hotNSLcqXCQNpNqqz6ZRvo8yTLpump" # @param {type: "string"}
base_ca = "0x23dD3Ce6161422622E773E13dAC2781C7f990D45" # @param {type: "string"}
arb_ca = "0x37a645648dF29205C6261289983FB04ECD70b4B3" # @param {type: "string"}

ca_to_buy = solana_ca

In [ ]:
value_usd = 1.0 # @param {type: "number"}
quantity = None # @param {type: "number"}
slippage = 1  # @param {type: "number"}
task_ = disciple.schedule_buy(disciple_agent_id, ca_to_buy, quantity, value_usd, slippage)

In [ ]:
status = disciple.check_task_status(task_['task_id'])

pprint(status)

## 9. Get the decisions of the Agent

In [29]:
response = disciple.get_decisions(disciple_agent_id)

pprint(response)

[]


# Disciple Lifecycle APIs

## 10. Update a Disciple

In [ ]:
disciple_id = "" # @param {type: "string"}
name = "Disciple 420" # @param {type: "string"}
description = "A loyal and trusted disciple of MONK" # @param {type: "string"}
persona = "A loyal and trusted disciple of MONK" # @param {type: "string"}
data_source = "trending" # @param ["trending", "latest"]
decision_prompt_pool = "You are a pro memecoin trader on Solana. Use your understanding of memcoins, market psychology and on-chain metrics to identify coins with high growth potentials.  MISSION: Make an informed and calculated judgement on weather to buy, hold, sell or pass on a memecoin.  GOAL: Your GOAL is to generate consistent profits over a **longer-term period (3 - 7 days)** by blending **momentum trading, volatility management, and adaptive feedback loops**.  STRATEGY: ### 1. **Initial Assessment for Entry**    - **Assess Pool Age:**      - If the pool was created recently (within 1-2 days), it suggests early-stage momentum. Prioritize memecoins that show strong early trading interest, as indicated by high **volume in the first 24 hours**.     - **Check FDV vs. Market Cap Ratio:**      - **FDV close to Market Cap:** Indicates potential bullishness with less risk of dilution. This supports entering a longer-term trade.      - **FDV significantly higher than Market Cap:** Indicates risk of dilution; proceed cautiously with tighter risk management.     - **Review Price Change Trends (up to 24hr):**      - Enter trades where there is a **consistent upward price change** across intervals (5m, 1hr, 6hr, 24hr), supported by rising volume.      - For a **bullish entry**, ensure positive price change across multiple intervals, particularly in 1hr, 6hr, and 24hr.  ### 2. **Incorporate Volatility Analysis**    - **Use ATR to Define Entry and Exit Ranges:**      - Calculate the **Average True Range (ATR)** to set entry and stop-loss levels. Wider ATR suggests higher potential rewards, but also increased risk.      - Set initial stop-loss levels at **1.5-2x the ATR** to avoid premature exits due to regular volatility.     - **Volatility Breakout Entry:**      - Enter long positions when the price breaks above recent volatility bands (e.g., Bollinger Bands), accompanied by strong volume over 6hr intervals.      - Avoid entering trades during extreme volatility spikes unless there is consistent follow-through in volume and price over longer intervals (6hr or more).  ### 3. **Adaptive Position Sizing**    - **Start with Smaller Initial Positions:**      - Given the high volatility, begin with a smaller position size to minimize risk. As the trend confirms (positive price and volume over 12hr), gradually increase the position size.     - **Adjust Position Based on Recent Trade Success:**      - If recent trades have been profitable, **increase position size by 10-20%** to capitalize on momentum.      - If recent trades have had high losses, **reduce position size** and tighten stops to avoid further losses.  ### 4. **Feedback Loop with Last Trades**    - **Analyze Trade Patterns:**      - Review the **last 5 trades** to identify common triggers for success or failure (e.g., rapid volume surges, resistance breakouts, or social media-driven momentum).      - If the **last profitable trades** had similar conditions (e.g., positive price change after volume spikes), replicate those entry conditions for prolonged trades.     - **Use Historical Exit Points for Guidance:**      - If past trades reached a **profit target of 10-20%** within 12-24 hours, set similar targets and be ready to adjust based on current momentum.      - If the last loss resulted from holding too long despite declining volume, be prepared to exit sooner in similar conditions.  ### 5. **Mid-Trade Management for 12-36 Hours**    - **Reassess at 12hr and 24hr Intervals:**      - Every 12 hours, evaluate whether the momentum is sustained: check volume, price change, and transaction patterns. If the volume remains strong and price trends are positive, maintain the position.     - **Trailing Stop-Loss:**      - Implement a **trailing stop-loss** based on the 6hr ATR to lock in profits as the price moves up. This allows you to capture gains while keeping the potential for further upside.      - Adjust the trailing stop if momentum remains strong after 24 hours, allowing for an extended hold.  ### 6. **Longer-Term Hold Strategy (Up to 7 Days)**    - **Monitor 24hr and 6hr Volume Trends:**      - If 24hr volume continues to rise, hold the position for up to 3 days to maximize gains.      - For holds longer than 3 days, ensure consistent positive price change in the 24hr interval and sustained volume increases.     - **Identify Extended Trends or Reversals:**      - Use **12hr and 24hr moving averages (MA)** as benchmarks:        - If the price consistently stays above the 12hr MA, it indicates sustained bullish momentum—hold longer.        - If the price drops below the 12hr MA, consider reducing position size or exiting partially.  ### 7. **Exit Criteria for 3-7 Days**    - **Profit Targets:**      - Set multiple profit targets (e.g., at +10%, +20%, and +30%).      - Use the **last 5 successful trades** to determine the most effective profit levels.     - **Volatility and Volume Drop:**      - Exit if there’s a significant **drop in 6hr or 24hr volume**, as it signals a decline in momentum.      - Exit earlier if there’s a sudden spike in **sell transactions** over 6hr intervals, indicating a potential trend reversal.    - **OHLCV Analysis:**     - If the **Open-High-Low-Close-Volume (OHLCV)** pattern shows consistent lower highs and lower lows over 24hr intervals, consider exiting to avoid further losses.     - If the OHLCV pattern remains bullish with higher highs and higher lows, consider holding the position for an extended period.     - You have OHLCV data in durations of 1 minute, 1 hour and 1 day. Access all of them carefully.  ### Summary of Strategy Workflow 1. **Entry:**    - Pool age, FDV vs. Market Cap, consistent positive price changes, and volatility breakout.  2. **Position Sizing:**    - Start small, scale up as momentum confirms, and adapt based on recent trades.  3. **Mid-Trade Management:**    - Reassess every 12hr, use trailing stops, and adjust based on ATR.  4. **Longer Hold:**    - Use 24hr volume and MA trends to sustain the trade.  5. **Exit:**    - Based on profit targets, volume drops, increased sell transactions, or external events." # @param {type: "string"}
decision_prompt_portfolio = "You are a seasoned crypto memecoin portfolio manager on Solana. Your only focus and job is keep increasing profits for yourself.  You are managing a portfolio of memecoins.  You are about to take following decision on your portfolio. You must analyze each of them critically. Specially look at the amount you're about to invest in each coin. Look at the coin's FDV and mcap and analyse if it's the best way to invest.  Also, you can rebalance your portfolio by selling some coins and buying new ones. You can also hold some coins if you think they have potential to grow further." # @param {type: "string"}
twitter_username = "0xmonk" # @param {type: "string"}
fc_username = "0xmonk" # @param {type: "string"}
max_investment_per_session = 0.2 # @param {type:"slider", min:0, max:1, step:0.01}
stop_loss = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
trailing_stop_loss = 0.3 # @param {type:"slider", min:0, max:1, step:0.01}
take_profit = 1.0 # @param {type:"number"}

# @markdown ### Simulation specific settings
simulation = True # @param {type: "boolean"}
simulation_initial_usd = 1000 # @param {type: "number"}
investment_for_buy = None # @param {type: "number"}
investment_for_buy_usd = 100.0 # @param {type: "number"}

In [ ]:
# Set the agent to OnChain Mode
disciple_id = "2428457f-3cae-4ee7-b8d5-876bb06d00d9" # @param {type: "string"}

updates = {
    "dry_run": False,
    "mode": 1,
    "investment_for_buy_usd": investment_for_buy_usd
}

disciple.update_agent(disciple_id, **updates)

### 10.1. Pause an agent

This code shows how to pause the agent. In order to unpause the agent, simply pass `is_active` as `True`.

In [ ]:

disciple_id = "2428457f-3cae-4ee7-b8d5-876bb06d00d9" # @param {type: "string"}

updates = {
    "is_active": False,
}

disciple.update_agent(disciple_id, **updates)

### 10.2. Update the parents of an Agent using the update endpoint

In this endpoint, the existing parents of the agent are replaced with the new parents. This should be used with caution.
In more practical scenarios, you would want to use the `add_parents` and `delete_parents` calls to add or remove parents from the agent.

In [ ]:
disciple_id = "b48591bd-54fa-4d8d-aa20-98d3c4d9e1d6" # @param {type: "string"}
parent_ids = ["1857242a-e3df-4fc9-8de5-2a111663789c"]

updates = {
  "parents": parent_ids
}

res = disciple.update_agent(disciple_id, **updates)
pprint(res)

## 11. Add parents to an agent

In [ ]:
disciple_id = "0133584a-c87b-4b85-b8de-6aa9172e3628" # @param {type: "string"}
parent_ids = ["1857242a-e3df-4fc9-8de5-2a111663789c"]

res = disciple.add_parents(disciple_id, parent_ids)
pprint(res)

## 12. Remove parents from an agent

In [ ]:
disciple_id = "b48591bd-54fa-4d8d-aa20-98d3c4d9e1d6" # @param {type: "string"}
parent_ids = ["1857242a-e3df-4fc9-8de5-2a111663789c"]

res = disciple.delete_parents(disciple_id, parent_ids)
pprint(res)

## 13. Delete the Agent

Deleting is permanent. All the keys and data will be deleted. Before deleting, make sure the user has backed up the keys and wallets.

In [ ]:
disciple_id = "" # @param {type: "string"}

res = disciple.delete_disciple(disciple_id)
pprint(res)

## 14. Get an agent details

Gets all the details (metadata) of an agent.

In [39]:
disciple_id = "83a3232a-a7e1-4b25-b8fc-f09d2e58b571" # @param {type: "string"}
res = disciple.get_agent(disciple_id)
pprint(res)

{'address': 'FCM9FQwU4XpqLTfB5QgscNmKjZwfCkM54YFAV1zcLFtD',
 'chain': 'SOLANA',
 'data_source': 'trending',
 'decision_prompt_pool': None,
 'decision_prompt_portfolio': None,
 'description': 'A loyal and trusted disciple of MONK',
 'dry_run': False,
 'dry_run_initial_usd': 14.432,
 'fc_username': '0xmonk',
 'id': '83a3232a-a7e1-4b25-b8fc-f09d2e58b571',
 'investment_for_buy': None,
 'investment_for_buy_usd': 10.0,
 'is_active': True,
 'max_investment_per_session': None,
 'mode': 0,
 'name': 'Aron Alpha Agent',
 'parents': [],
 'persona': 'A loyal and trusted disciple of MONK',
 'stop_loss': 0.5,
 'take_profit': 1.0,
 'trailing_stop_loss': 0.3,
 'twitter_username': '0xmonk',
 'user_id': 'd9a486fa-8dd0-4593-b2f6-9443d3a0abd6'}


## 15. Sync the agent with onchain values

In [27]:
disciple_id = "82bff902-88ad-43ea-a8e2-433c1f899e1a" # @param {type: "string"}
task_id = disciple.schedule_sync(disciple_id)

pprint(task_id)

{'task_id': '8a41c0bf-383d-411c-bbe5-757438bbeb5e'}


In [23]:
status = disciple.check_task_status(task_id)

pprint(status)

{'result': None,
 'status': 'PENDING',
 'task_id': "{'task_id': '0289aaeb-d88a-45e0-88f2-e5b59f31f59f'}"}
